## Open Weather Map API

Purpose is to retrieve data from Open Weather Map API (https://openweathermap.org/api/one-call-3#start) for building Ciampino Digital Model and getting weather data for it.

### Setting API Key

Here is reported API Key for performing calls.

In [1]:
API_KEY = '997b847a9ed5dda58598e1f0015be1e6'
CITY = 'Ciampino,Lazio,IT'

Then, it's used Geocoding API from Open Weather Map to retrieve latitude and longitude of inspected city, which is Ciampino.
But first, import all necessary libraries.

In [2]:
import requests
import datetime
import pandas as pd

In [3]:
# Geocoding API Url

def geocode_city():
    return f'http://api.openweathermap.org/geo/1.0/direct?q={CITY}&limit=5&appid={API_KEY}'

In [4]:
# Data retrieved from Geocoding API url and then latitude and longitude stored
def get_city_data():
    res = requests.get(geocode_city()).json()
    return {'lat': res[0]['lat'], 'lon': res[0]['lon']}

city_lat = get_city_data()['lat']
city_lon = get_city_data()['lon']

Now it's possible to make first call with Open Weather Map API for weather.

In [5]:
def get_weather_data(url):
    return requests.get(url).json()

In [7]:
# API for Current Weather Data ONLY!
url_curr_weather_api = f'https://api.openweathermap.org/data/2.5/weather?lat={city_lat}&lon={city_lon}&&appid={API_KEY}&units=metric'
print(get_weather_data(url_curr_weather_api))

{'coord': {'lon': 12.6005, 'lat': 41.8003}, 'weather': [{'id': 800, 'main': 'Clear', 'description': 'clear sky', 'icon': '01n'}], 'base': 'stations', 'main': {'temp': 11.75, 'feels_like': 10.76, 'temp_min': 10.5, 'temp_max': 12.88, 'pressure': 1024, 'humidity': 68, 'sea_level': 1024, 'grnd_level': 1007}, 'visibility': 10000, 'wind': {'speed': 1.54, 'deg': 60}, 'clouds': {'all': 0}, 'dt': 1735402101, 'sys': {'type': 1, 'id': 6792, 'country': 'IT', 'sunrise': 1735367781, 'sunset': 1735400758}, 'timezone': 3600, 'id': 3178738, 'name': 'Ciampino', 'cod': 200}


In [6]:
from datetime import datetime
# Re-elaborate data from API response for setting correct date (not Unix timestamps)
def set_date(url: str):
    data = get_weather_data(url)
    print(data, type(data))
    if 'dt' in data:
        data['dt'] = datetime.fromtimestamp(data['dt']).strftime('%Y-%m-%d %H:%M:%S')
    if 'sys' in data:
        data['sys']['sunrise'] = datetime.fromtimestamp(data['sys']['sunrise']).strftime('%Y-%m-%d %H:%M:%S')
        data['sys']['sunset'] = datetime.fromtimestamp(data['sys']['sunset']).strftime('%Y-%m-%d %H:%M:%S')
    return data

In [9]:
print(set_date(url_curr_weather_api))

{'coord': {'lon': 12.6005, 'lat': 41.8003}, 'weather': [{'id': 800, 'main': 'Clear', 'description': 'clear sky', 'icon': '01n'}], 'base': 'stations', 'main': {'temp': 11.75, 'feels_like': 10.76, 'temp_min': 10.5, 'temp_max': 12.88, 'pressure': 1024, 'humidity': 68, 'sea_level': 1024, 'grnd_level': 1007}, 'visibility': 10000, 'wind': {'speed': 1.54, 'deg': 60}, 'clouds': {'all': 0}, 'dt': '2024-12-28 17:08:21', 'sys': {'type': 1, 'id': 6792, 'country': 'IT', 'sunrise': '2024-12-28 07:36:21', 'sunset': '2024-12-28 16:45:58'}, 'timezone': 3600, 'id': 3178738, 'name': 'Ciampino', 'cod': 200}


In [7]:
from datetime import timedelta

def increase_date(start_date: int, limit_date: int, increment_days: int):
    # Convert human date in timestamps for starting and limit date
    # Return the incremented date when both are known, else returns start or limit date
    new_date: int
    if limit_date:
        if start_date < limit_date:
            new_date = increase_unix_timestamp(start_date, increment_days)
            return new_date
        else:
            return limit_date
    else:
        return start_date

def date_str_to_unix_timestamp(date_str: str):
    # Convert the string date to a datetime object
    return int(datetime.strptime(date_str, '%Y-%m-%d').timestamp())

def increase_unix_timestamp(unix_timestamp, days):
    # Convert Unix timestamp to datetime
    # Add days and then convert datetime object in timestamp
    dt = datetime.fromtimestamp(unix_timestamp)
    new_dt = dt + timedelta(days=days)
    return int(new_dt.timestamp())

def unix_to_date(unix_timestamp):
    # Convert Unix timestamp to datetime object
    # Format the datetime object as 'YYYY-MM-DD'
    return datetime.fromtimestamp(unix_timestamp).strftime('%Y-%m-%d')

In [8]:
from pandas import DataFrame

def map_dataframe(data):
    return DataFrame({
        'date': [data['date']],
        'tz': [data['tz']],
        'lat': [data['lat']],
        'lon': [data['lon']],
        'temp_min': [data['temperature']['min']],
        'temp_max': [data['temperature']['max']],
        'temp_1200': [data['temperature']['afternoon']],
        'temp_0000': [data['temperature']['night']],
        'temp_1800': [data['temperature']['evening']],
        'temp_0600': [data['temperature']['morning']],
        'cloud_cover': [data['cloud_cover']['afternoon']],
        'humidity': [data['humidity']['afternoon']],
        'precipitation_tot': [data['precipitation']['total']],
        'pressure': [data['pressure']['afternoon']],
        'wind_max_speed': [data['wind']['max']['speed']],
        'wind_direction': [data['wind']['max']['direction']],
    })

In [9]:
# https://openweathermap.org/api/one-call-3#history_daily_aggregation

def get_info(start_date: str, limit_date: str, increment_days: int):
    start_date_unix = date_str_to_unix_timestamp(start_date)
    limit_date_unix = date_str_to_unix_timestamp(limit_date)
    data_list = []  # Store data to append later
    while start_date_unix <= limit_date_unix:
        new_date = increase_date(start_date_unix, limit_date_unix, increment_days)
        new_date = unix_to_date(new_date)
        weather_data = set_date(f'https://api.openweathermap.org/data/3.0/onecall/day_summary?lat={city_lat}&lon={city_lon}&date={new_date}&appid={API_KEY}&units=metric')
        # Map data and collect it
        mapped_data = map_dataframe(weather_data)
        data_list.append(mapped_data)
        
        # Update start_date_unix to avoid infinite loop
        start_date_unix = date_str_to_unix_timestamp(new_date)
    return pd.concat(data_list, ignore_index=True)

In [15]:
def save_yearly_df(df: DataFrame, year: str):
    year_df = pd.DataFrame()
    year_df = df
    year_df.to_csv(f'ciampino_weather_{year}.csv')
    if df.size == year_df.size:
        print(f'Dataframe for year {year} created. Check in \'ciampino_weather_{year}.csv\'')
    else:
        print('Something is wrong')
    
df = pd.DataFrame()
df = get_info('2011-12-31', '2012-12-31', 1)
save_yearly_df(df, '2012') 

{'cod': 429, 'message': 'Your account is temporary blocked due to exceeding of requests limitation of your subscription type. Please choose the proper subscription https://openweathermap.org/price'} <class 'dict'>


KeyError: 'date'

In [48]:
import time

def measure_execution_time(func, *args, **kwargs):
    start_time = time.perf_counter()  # Start the timer
    result = func(*args, **kwargs)    # Run the function
    end_time = time.perf_counter()    # End the timer
    
    elapsed_time = end_time - start_time  # Calculate elapsed time
    print(f"Execution time: {elapsed_time:.6f} seconds")
    return result
